In [1]:
# specify location of `msd_summary_file.h5` or the whole `*.tar.gz` file here!

# location = '../millionsongsubset_full.tar.gz'
location = '../msd_summary_file.h5'

In [2]:
# imports:

import numpy as np
import pandas as pd
import tarfile
import itertools as it
from sklearn import preprocessing

## DuplicateDetector class
This class largely consists of boilerplate code, such as the initialization routine and the possibility for the user to change parameters like the amount `N` of dataset items to include, the rows `r` and bins `b` for the hash function to use, which features to include in the comparison, and what cutoff to use for the cosine similarity. There are also two simple methods `naive_cosine_similarity()` and `check_pairs()` that loop through all pairs or candidate pairs, respectively, and call the method `cosine_similarity(pair)`, which simply returns the dot product of the pair divided by the product of their norms. 

The important part of the class is contained in the three methods described in more detail here:

### generate_signature_matrix()
A pseudo signature matrix is computed by generating `r*b` random vectors and multiplying them with the data matrix. We don't actually care about the magnitude of these values, so the sign is computed. For simplicity, negative values are also converted to zero, so that our signature matrix consists of {0,1} entries rather than {-1,1} entries (This also eliminates the miniscule risk of getting a ternary matrix with {-1,0,1} entries from the sign function).

Finally, we want to split the signature matrix into b bins, so it's reshaped from an `N` x `(r*b)` matrix into a `b` x `N` x `r` tensor, consisting of `b` different `N` x `r` sub-matrices - one for each bin.

### get_candidate_pairs()
This method iterates through all `b` bins and finds samples that have an identical hash within the bin. In order to achieve this quickly, numpy routines are used, instead of iterating through the whole array. As this is a bit tricky and involves the manipulation of data types for the sake of memory level access (something more associated with C than with Python programming), the individual steps are described here.

1. ```python 
sub_matrix = np.ascontiguousarray(self.sm[b]).view(np.dtype((np.void, self.sm.dtype.itemsize * self.r)))
```
First create a local copy of the `b`-th sub-matrix, which is an `N` x `r` array of 8-bit unsigned integers. By calling the `view(dtype)` method, we are telling numpy to instead view it as a 1-dimensional size `N` array of (`8*r`)-bit voids. The reason for this is straightforward: numpy has an extremely efficient implementation for sorting and finding duplicates in `numpy.unique()`, but this only works on 1-dimensional or flattened arrays.

2. ```python
_, inverses = np.unique(sub_matrix, return_inverse=True)
```
Using the `return_inverse` flag, `numpy.unique()` returns not only an array `_` of all unique entries, but also an array `inverses` indexing which unique entry appears at each position in the original array. Mathematically speaking, this is a bijective function $\mathcal{F} : \mathbb{B}^r \rightarrow \mathbb{N}_0$. We are transforming from `r`-dimensional Boolean vectors to sequential integers, which are easier to work with. Effectively, `inverses` is a hash of our hash, so that it fits into convential data types.

3. ```python
sorted_args = np.argsort(inverses)
```
If we argsort this array, we now have the indices of our samples (integers from 0 to `N-1`) arranged in an order so that all samples with identical hashes are adjecent to each other.

4. ```python
sub_matrix_sorted = inverses[sorted_args]
_, indices = np.unique(sub_matrix_sorted, return_index=True)
candidate_tuples = np.split(sorted_args, indices[1:])
```
This array of `sorted_args` is now split up into several sub-arrays, each containing only samples with identical hashes. These subarrays could be of size 1 (no other matching sample), size 2 (exactly two matching samples), or size >2 (many samples all with the same hash).

5. ```python
for q in candidate_tuples:
    if q.size > 2:
        candidate_pairs.extend([np.asarray(g) for g in it.combinations(q, 2)])
    elif q.size == 2:
        candidate_pairs.append(q)
```
So finally we loop through these these tuples, discarding the unmatched samples, and using `itertools.combinations()` to generate all pairwise combinations within the >2 tuples. This concludes the generation of candidate pairs for a single bin `b`.

### prune_duplicate_pairs(candidate_pairs)

As we treated each bin totally separately, it is possible that some candidate pairs were detected in several bins. The last step before returning the candidate pairs is to prune duplicates:

```python
pc = np.ascontiguousarray(np.sort(candidate_pairs), dtype='uint32').view(dtype='uint64')
self.candidate_pairs = np.unique(pc).view(dtype='uint32').reshape(-1, 2)
```
This uses a similar trick to above. The candidate pairs are saved as a `P` x `2` array of 32-bit unsigned integers (with `P` being the amount of pairs found), but numpy is told to view them as a 1-D size `P` array of 64-bit unsigned integers. Duplicates can now be trivially and efficiently removed using `numpy.unique()` and then the resulting shorter array is transformed and reshaped back to a `Q` x `2` array of 32-bit uints.


In [3]:
class DuplicateDetector:
    """
    Class to detect duplicates in the million song dataset.

    Accepts user parameters:
    int N: amount of items from the dataset to consider
    int r: amount of rows per bin of the hash function (must be below 64 due to data representation)
    int b: amount of bins in the hash function
    list of strings features: the names (keys) of the features to include in the similarity analysis
    string location: the location of the .tar.gz archive containing the songs
    float epsilon: the maximum angle between vectors that will be considered a duplicate
    float distance: maximum cosine distance for duplicates (if both epsilon and distance are specified,
                    distance takes priority.

    Allows the following methods:
    add_feature(feature): adds an extra feature to the list of features, reloads the data matrix
    remove_feature(feature): removes one of the features from the list of features, reloads the data matrix
    set_epsilon(epsilon): change the cutoff angle for the similarity comparison
    set_distance(distance): change the cutoff cosine distance for the similarity comparison
    generate_signature_matrix(): generates the signature matrix with random hyperplanes
    get_candidate_pairs(): generates all candidate pairs that match from the hashing of the signature matrix
    check_pairs(): checks all of the candidate pairs to compute the true cosine similarity, selects those <epsilon
    list naive_cosine_similarity(): like above, but checks all possible pairs, without any hashing.
    """

    ### Initialization and User Interface functions:

    def __init__(self, N=10000, r=64, b=3, features=[], location='./millionsongsubset_full.tar.gz',
                 epsilon=None, distance=None):
        self.location = location
        self.r = r
        self.b = b
        self.N = N
        self.overhead_table = self.get_overhead_table()
        self.features = []
        for feature in features:
            if self.legitimate_feature(feature) and feature not in self.features:
                self.features.append(feature)
        if not self.features:
            self.features = ['duration', 'end_of_fade_in', 'key', 'loudness', 'mode',
                             'start_of_fade_out', 'tempo', 'time_signature']
        if len(self.overhead_table[self.features[0]]) < self.N:
            self.N = len(self.overhead_table[self.features[0]])
            print('maximum N value exceeded, correcting to {}.'.format(self.N))
        self.data_matrix = self.get_data_matrix()
        self.actual_pairs = []
        if not (epsilon and distance):
            self.cutoff = np.cos(5.0*np.pi/180.0)
        if distance:
            self.cutoff = 1 - distance
        if epsilon:
            self.cutoff = np.cos(epsilon * np.pi / 180.0)

    def get_data_matrix(self):
        M = np.zeros([self.N, len(self.features)])
        for f, feature in enumerate(self.features):
            M[:, f] = self.overhead_table[feature][:self.N]
        return preprocessing.scale(M)

    def get_overhead_table(self):
        if self.location.endswith('.tar.gz'):
            tar = tarfile.open(self.location, 'r')
            members = tar.getmembers()
            tar.extract(members[5])
            summary = pd.HDFStore(members[5].name)
            return summary['/analysis/songs']
        elif self.location.endswith('.h5'):
            summary = pd.HDFStore(self.location)
            return summary['/analysis/songs']

    def reload_data(self):
        self.data_matrix = self.get_data_matrix()

    def add_feature(self, feature):
        if self.legitimate_feature(feature) and feature not in self.features:
            self.features.append(feature)
            self.reload_data()

    def remove_feature(self, feature):
        if feature in self.features:
            self.features.remove(feature)
            self.reload_data()

    def legitimate_feature(self, feature):
        return True if feature in self.overhead_table.keys() else False

    def set_epsilon(self, epsilon):
        self.cutoff = np.cos(epsilon * np.pi / 180.0)

    def set_distance(self, distance):
        self.cutoff = 1 - distance

    def set_N(self, N):
        self.N = N
        if len(self.overhead_table[self.features[0]]) < self.N:
            self.N = len(self.overhead_table[self.features[0]])
            print('maximum N value exceeded, correcting to {}.'.format(self.N))
        self.reload_data()

    ### Similarity / Distance functions:

    def generate_signature_matrix(self):
        # multiply features with random hyperplanes:
        signature_matrix = self.data_matrix.dot(np.random.normal(0, 1, [len(self.features), self.r*self.b]))
        # convert all non-positive values to 0, all positive values to 1:
        binary_signature_matrix = np.maximum(np.sign(signature_matrix), 0, signature_matrix).astype('uint8')

        # reshape the "N x (r*b)" signature matrix into a "b x N x r" signature tensor (consisting of b separate
        # signature sub-matrices that we can individually search for duplicates):
        self.sm = np.reshape(binary_signature_matrix, [self.N, self.b, self.r]).transpose(1, 0, 2)

    def get_candidate_pairs(self):
        candidate_pairs = []
        # loop through our b different sub-matrices:
        for b in range(self.b):

            # find all identical entries from the b'th signature sub-matrix using some dtype trickery:
            sub_matrix = np.ascontiguousarray(self.sm[b]).view(np.dtype((np.void, self.sm.dtype.itemsize * self.r)))
            _, inverses = np.unique(sub_matrix, return_inverse=True)
            sorted_args = np.argsort(inverses)
            sub_matrix_sorted = inverses[sorted_args]
            _, indices = np.unique(sub_matrix_sorted, return_index=True)
            candidate_tuples = np.split(sorted_args, indices[1:])

            # each entry of candidate_tuples contains the indices of samples with the same hash. The size of the entries
            # may be anywhere between 1 and several hundred, so throw out the size=1 entries, and expand the size>2
            # entries to all pairwise combinations of their members:
            for q in candidate_tuples:
                if q.size > 2:
                    candidate_pairs.extend([np.asarray(g) for g in it.combinations(q, 2)])
                elif q.size == 2:
                    candidate_pairs.append(q)
        self.prune_duplicate_pairs(candidate_pairs)

    def prune_duplicate_pairs(self, candidate_pairs):
        # eliminate duplicate detections (that were marked identical from several sub-matrices)
        pc = np.ascontiguousarray(np.sort(candidate_pairs), dtype='uint32').view(dtype='uint64')
        self.candidate_pairs = np.unique(pc).view(dtype='uint32').reshape(-1, 2)

    def cosine_similarity(self, pair):
        return np.dot(self.data_matrix[pair[0]], self.data_matrix[pair[1]]) / (np.linalg.norm(self.data_matrix[pair[0]]) * np.linalg.norm(self.data_matrix[pair[1]]))

    ### Simple for-loop functions computing the similarity between samples:

    def check_pairs(self):
        for pair in self.candidate_pairs:
            if self.cosine_similarity(pair) > self.cutoff:
                self.actual_pairs.append(pair)
        self.actual_pairs = np.asarray(self.actual_pairs)

    def naive_cosine_similarity(self):
        naive_pairs = []
        for n in range(self.N):
            for m in range(n+1, self.N):
                if self.cosine_similarity([n, m]) > self.cutoff:
                    naive_pairs.append([n, m])
        return naive_pairs

In [4]:
# now let's see how fast it is:

import time

dd = DuplicateDetector(N=10000, r=64, b=3, epsilon=2, location=location)

start = time.time()
dd.generate_signature_matrix()
dd.get_candidate_pairs()
print('Generated signature matrix, found {} candidate pairs in {:.2}s.'.format(len(dd.candidate_pairs), time.time()-start))

start = time.time()
dd.check_pairs()
print('Found {} actual pairs in {:.2}s.'.format(len(dd.actual_pairs), time.time()-start))

'''
As reference, on a MacBook Air with i7 processor, the code obtains the results:
> Generated signature matrix, found 4256 candidate pairs in 0.13s.
> Found 23 actual pairs in 0.053s.
'''

Generated signature matrix, found 4256 candidate pairs in 0.13s.
Found 23 actual pairs in 0.053s.


In [5]:
# for comparison, the naive pairwise comparison of everything:

start = time.time()
p = dd.naive_cosine_similarity()
print('found {} actual pairs naively in {:.4}s.'.format(len(p), time.time()-start))

'''
As reference, on a MacBook Air with i7 processor, the code obtains the results:
> found 23 actual pairs naively in 607.0s.
'''

found 23 actual pairs naively in 607.0s.
